# Transit Access for Equaity Emphasis Areas #

_(RN)_

Areas that are identified as Equity Emphasis Areas (EEAs) that are considered transit-viable, and are
underserved by public transit.  This is calculated in 3 broad steps:
  * Calculate Equity Emphasis Areas (EEA)
  * Calculate Transit Viability
  * Calculate Areas Underserved by Transit

In [7]:
import arcpy
import os
import pandas as pd

arcpy.env.overwriteOutput = True

main_path = os.path.dirname(os.path.abspath(''))
common_datasets_gdb = os.path.join(main_path, r'A1 - Common Datasets\Common_Datasets.gdb')

### Calculate EEAs ###

Threshold for Need for Transit Access for Equity Emphasis Areas: Transit Viable Areas are those that have a level population density matching at least the bottom 10th percentile density of areas that are currently served by transit.

The first step is to download the required data from ACS using the CreateTables.py and StatsDict.py scripts.  The results should be stored in EEA_Statistics.csv.

In [8]:
regional_networks = f'{common_datasets_gdb}\\RegionalNetworks_NoOverlap'
source_block_groups = f'{common_datasets_gdb}\\Block_Group'
intermediate_gdb = f'{main_path}\\Need for Transit Access for Equity Emphasis Areas\\data\\intermediate.gdb'
output_gdb = f'{main_path}\\Need for Transit Access for Equity Emphasis Areas\\data\\output.gdb'

# Create gdbs if do not exist
for gdb_path in [intermediate_gdb, output_gdb]:
    if os.path.exists(os.path.dirname(gdb_path)):
        if not os.path.exists(gdb_path):
            arcpy.management.CreateFileGDB(os.path.dirname(gdb_path), os.path.basename(gdb_path))
    else:
        raise Exception(f'Path for GDB does not exist: \n{os.path.dirname(gdb_path)}')

fc_block_groups = os.path.join(intermediate_gdb, 'Block_Group')  # Copy of block groups that will be placed in intermediate.gdb
arcpy.FeatureClassToFeatureClass_conversion(source_block_groups, os.path.dirname(fc_block_groups), os.path.basename(fc_block_groups))

# The steps listed here are accomplished in CreateTables.py and saved as EEA_Statistics.csv:
# Collect resident data on income, age, race and ethnicity, English proficiency, disability, and total population.
# Calculate from US Census ACS tables the number of residents whose income is below 150% of the poverty level, who are
# 75 or older, who belong to a racial minority, who are Hispanic/Latino, and who do not speak English “very well.”

# Disability data at the tract level are downloaded as EEA_Statistics_Disability.csv


# Load EEA Statistics and join the disability data from the tract level to the block group level
EEA_statistics = 'EEA_Statistics.csv'
df_EEA_statistics = pd.read_csv(EEA_statistics)

# Replace census field name codes with readable names
from StatsDict import ACS_Tables
fieldsDict = ACS_Tables[0]['data']
df_EEA_statistics.rename(columns=fieldsDict, inplace=True)

# Load disability statistics
disability_statistics = 'EEA_Statistics_Disability.csv'
df_disability_statistics = pd.read_csv(disability_statistics)
disability_columns = {
    'S0101_C01_001E': 'Total Population',
    'S1810_C01_001E': 'Total civilian noninstitutionalized population',
    'S1810_C02_001E': 'Civilian noninstitutionalized population with a disability',
    'S1810_C03_001E': 'Percent with a disability'
}
df_disability_statistics.rename(columns=disability_columns, inplace=True)
# Fix data issue where too-low to measure values are provided as -66666666
# df_disability_statistics.loc[df_disability_statistics['Percent with a disability'] < 0, 'Percent with a disability'] = 0
df_disability_statistics['Percent with a disability'] = (df_disability_statistics['Civilian noninstitutionalized population with a disability'] / df_disability_statistics['Total Population']) * 100

# Create unique ID by tract to join disability data to rest of statistics
df_EEA_statistics['countytract'] = df_EEA_statistics['county'].astype(str) + '.' + df_EEA_statistics['tract'].astype(str)
df_disability_statistics['countytract'] = df_disability_statistics['county'].astype(str) + '.' + df_disability_statistics['tract'].astype(str)

# Reduce disability dataframe to two necessary fields
df_disability_statistics = df_disability_statistics[['countytract', 'Percent with a disability']]



In [9]:
# Add disability data to EEA_statistics dataframe and calculate disability population
df_EEA_statistics = df_EEA_statistics.merge(df_disability_statistics, on='countytract')
df_EEA_statistics['Residents with a disability'] = round((df_EEA_statistics['Percent with a disability']/100) * df_EEA_statistics['Total Population'])

# Find share of population for low income, > age 75, racial minority, hispanic/latino, limited english proficiency,
# and disability population
df_EEA_statistics['poverty_ratio_sum_share'] = df_EEA_statistics['poverty_ratio_sum'] / df_EEA_statistics['Total Population']
df_EEA_statistics['over_75_sum_share'] = df_EEA_statistics['over_75_sum'] / df_EEA_statistics['Total Population']
df_EEA_statistics['total_minority_share'] = df_EEA_statistics['total_minority'] / df_EEA_statistics['Total Population']
df_EEA_statistics['Total Hispanic or Latino_share'] = df_EEA_statistics['Total Hispanic or Latino'] / df_EEA_statistics['Total Population']
df_EEA_statistics['do_not_speak_english_very_well_share'] = df_EEA_statistics['do_not_speak_english_very_well'] / df_EEA_statistics['Total Population']
df_EEA_statistics['Residents with a disability_share'] = df_EEA_statistics['Residents with a disability'] / df_EEA_statistics['Total Population']

In [10]:
# Locate the nearest regional network for each block group to calculate the regional averages

# Convert block groups to points
tbl_block_groups = os.path.join(intermediate_gdb, 'tbl_block_groups')
arcpy.TableToTable_conversion(fc_block_groups, intermediate_gdb, 'tbl_block_groups')
arcpy.AddField_management(tbl_block_groups, 'centLat', 'DOUBLE')
arcpy.AddField_management(tbl_block_groups, 'centLON', 'DOUBLE')
with arcpy.da.UpdateCursor(tbl_block_groups, ['INTPTLAT', 'INTPTLON', 'centLat', 'centLon']) as cur:
    for row in cur:
        row[2] = float(row[0])
        row[3] = float(row[1])
        cur.updateRow(row)

fc_block_group_pts = os.path.join(intermediate_gdb, 'fc_block_group_pts')
arcpy.management.XYTableToPoint(tbl_block_groups, fc_block_group_pts, 'centLon', 'CentLat')

# Locate the nearest RN for each
arcpy.Near_analysis(fc_block_group_pts, regional_networks, method='GEODESIC')

<Result 'c:\\Users\\daniel.fourquet\\Documents\\Tasks\\VTrans Update\\Mid-Term Needs Clean\\Need for Transit Access for Equity Emphasis Areas\\data\\intermediate.gdb\\fc_block_group_pts'>

In [11]:
# Create FID/RN_Name dictionary and add RN_Name to block group points layer
rn_dict = {row[0]:row[1] for row in arcpy.da.SearchCursor(regional_networks, ['OID@', 'RN_Name'])}

arcpy.AddField_management(fc_block_group_pts, 'RN_Name', 'TEXT')
with arcpy.da.UpdateCursor(fc_block_group_pts, ['NEAR_FID', 'RN_NAME']) as cur:
    for row in cur:
        rn_name = rn_dict[row[0]]
        row[1] = rn_name
        cur.updateRow(row)

# Regional means should be calculated using only blocks that fall within RNs.  This step will identify those blocks

block_groups_in_rn = os.path.join(intermediate_gdb, 'block_groups_in_rn')
arcpy.PairwiseClip_analysis(fc_block_group_pts, regional_networks, block_groups_in_rn)
block_groups_in_rn_geoids = [row[0] for row in arcpy.da.SearchCursor(block_groups_in_rn, 'GEOID')]  # Get list of block groups in rn

arcpy.AddField_management(fc_block_group_pts, 'in_RN', 'SHORT')
arcpy.SelectLayerByLocation_management(fc_block_group_pts, 'INTERSECT', regional_networks)
with arcpy.da.UpdateCursor(fc_block_group_pts, ['GEOID', 'in_RN']) as cur:
    for row in cur:
        if row[0] in block_groups_in_rn_geoids:
            row[1] = 1
        else:
            row[1] = 0
        cur.updateRow(row)

In [12]:
# Add RN_Name to stats DataFrame
rn_name_fields = ['GEOID', 'RN_Name', 'in_RN']
df_rn_names = pd.DataFrame([row for row in arcpy.da.SearchCursor(fc_block_group_pts, rn_name_fields)], columns=rn_name_fields)
df_rn_names['GEOID'] = df_rn_names['GEOID'].astype('int64')

df_EEA_statistics = df_EEA_statistics.merge(df_rn_names, left_on='GEO_ID', right_on='GEOID')

In [13]:
# Set null values to 0
share_fields = ['poverty_ratio_sum_share',
    'over_75_sum_share',
    'total_minority_share',
    'Total Hispanic or Latino_share',
    'do_not_speak_english_very_well_share',
    'Residents with a disability_share']
df_EEA_statistics.loc[df_EEA_statistics['Total Population'] == 0, share_fields] = 0

In [14]:
# Calculate regional averages
df_EEA_Statistics_Regional = df_EEA_statistics.loc[df_EEA_statistics['in_RN'] == 1].groupby('RN_Name').mean()
regional_fields = [
    'GEO_ID',
    'poverty_ratio_sum_share',
    'over_75_sum_share',
    'total_minority_share',
    'Total Hispanic or Latino_share',
    'do_not_speak_english_very_well_share',
    'Residents with a disability_share'
    ]
df_EEA_Statistics_Regional = df_EEA_Statistics_Regional[regional_fields]
df_EEA_Statistics_Regional.to_csv('EEA_Statistics_Regional.csv')

In [15]:
# Merge regional stats to main stats dataframe by geoid
df_EEA_statistics = df_EEA_statistics.merge(df_EEA_Statistics_Regional, left_on='RN_Name', right_on='RN_Name', suffixes=('_local', '_regional'))
df_EEA_statistics.to_csv('EEA_Statistics_Merge.csv', index=False)

In [16]:
# Divide the Block Group’s concentration for each resident category by the regional concentration to calculate the ratio of
# concentration (ROC).
df_EEA_statistics['poverty_ratio_sum_ROC'] = df_EEA_statistics['poverty_ratio_sum_share_local'] / df_EEA_statistics['poverty_ratio_sum_share_regional']
df_EEA_statistics['over_75_sum_ROC'] = df_EEA_statistics['over_75_sum_share_local'] / df_EEA_statistics['over_75_sum_share_regional']
df_EEA_statistics['total_minority_ROC'] = df_EEA_statistics['total_minority_share_local'] / df_EEA_statistics['total_minority_share_regional']
df_EEA_statistics['Total Hispanic or Latino_ROC'] = df_EEA_statistics['Total Hispanic or Latino_share_local'] / df_EEA_statistics['Total Hispanic or Latino_share_regional']
df_EEA_statistics['do_not_speak_english_very_well_ROC'] = df_EEA_statistics['do_not_speak_english_very_well_share_local'] / df_EEA_statistics['do_not_speak_english_very_well_share_regional']
df_EEA_statistics['Residents with a disability_ROC'] = df_EEA_statistics['Residents with a disability_share_local'] / df_EEA_statistics['Residents with a disability_share_regional']

# Sum all six ROCs into an index, converting all ROCs above 3 to 3, low-income ROCs below 1 to 0, and ROCs for the
# other categories below 1.5 to 0.

# Convert all ROCs above 3 to 3
df_EEA_statistics.loc[df_EEA_statistics['poverty_ratio_sum_ROC'] >= 3, 'poverty_ratio_sum_ROC'] = 3
df_EEA_statistics.loc[df_EEA_statistics['over_75_sum_ROC'] >= 3, 'over_75_sum_ROC'] = 3
df_EEA_statistics.loc[df_EEA_statistics['total_minority_ROC'] >= 3, 'total_minority_ROC'] = 3
df_EEA_statistics.loc[df_EEA_statistics['Total Hispanic or Latino_ROC'] >= 3, 'Total Hispanic or Latino_ROC'] = 3
df_EEA_statistics.loc[df_EEA_statistics['do_not_speak_english_very_well_ROC'] >= 3, 'do_not_speak_english_very_well_ROC'] = 3
df_EEA_statistics.loc[df_EEA_statistics['Residents with a disability_ROC'] >= 3, 'Residents with a disability_ROC'] = 3

# low-income ROCs below 1 to 0
df_EEA_statistics.loc[df_EEA_statistics['poverty_ratio_sum_ROC'] < 1, 'poverty_ratio_sum_ROC'] = 0

# ROCs for the other categories below 1.5 to 0
df_EEA_statistics.loc[df_EEA_statistics['over_75_sum_ROC'] < 1.5, 'over_75_sum_ROC'] = 0
df_EEA_statistics.loc[df_EEA_statistics['total_minority_ROC'] < 1.5, 'total_minority_ROC'] = 0
df_EEA_statistics.loc[df_EEA_statistics['Total Hispanic or Latino_ROC'] < 1.5, 'Total Hispanic or Latino_ROC'] = 0
df_EEA_statistics.loc[df_EEA_statistics['do_not_speak_english_very_well_ROC'] < 1.5, 'do_not_speak_english_very_well_ROC'] = 0
df_EEA_statistics.loc[df_EEA_statistics['Residents with a disability_ROC'] < 1.5, 'Residents with a disability_ROC'] = 0

# Sum all six ROCs into an index
df_EEA_statistics['EEA_index'] = sum([df_EEA_statistics['poverty_ratio_sum_ROC'], df_EEA_statistics['over_75_sum_ROC'], df_EEA_statistics['total_minority_ROC'], df_EEA_statistics['Total Hispanic or Latino_ROC'], df_EEA_statistics['do_not_speak_english_very_well_ROC'], df_EEA_statistics['Residents with a disability_ROC']])

# Flag a Block Group as an EEA if either the ROC for low-income or disability is at least 1
df_EEA_statistics['EEA'] = 'NO'
# df_EEA_statistics.loc[(df_EEA_statistics['poverty_ratio_sum_ROC'] >= 1) | (df_EEA_statistics['Residents with a disability_ROC'] >= 1), 'EEA'] = 'YES'
# df_EEA_statistics.loc[(df_EEA_statistics['EEA_index'] >= 2) | ((df_EEA_statistics['poverty_ratio_sum_ROC'] >= 1) | (df_EEA_statistics['Residents with a disability_ROC'] >= 1)), 'EEA'] = 'YES'
df_EEA_statistics.loc[(df_EEA_statistics['EEA_index'] >= 2) & ((df_EEA_statistics['poverty_ratio_sum_ROC'] >= 1) | (df_EEA_statistics['Residents with a disability_ROC'] >= 1)), 'EEA'] = 'YES'

In [17]:
# Add statistics to block group feature class
fields_to_add = [
    'poverty_ratio_pop',
    'poverty_ratio_pop_share',
    'poverty_ratio_ROC',
    'over_75_pop',
    'over_75_pop_share',
    'over_75_ROC',
    'total_minority_pop',
    'total_minority_pop_share',
    'total_minority_ROC',
    'total_hispanic_latino_pop',
    'total_hispanic_latino_pop_share',
    'total_hispanic_latino_ROC',
    'limited_english_proficiency_pop',
    'limited_english_proficiency_pop_share',
    'limited_english_proficiency_ROC',
    'disability_pop',
    'disability_pop_share',
    'disability_ROC',
    'eea_index',
    'eea',
    'in_RN',
    'total_pop'
]

for field in fields_to_add:
    fieldType = 'TEXT' if field == 'eea' else 'DOUBLE'
    print(f'Adding field {field} ({fieldType})')
    arcpy.AddField_management(fc_block_groups, field, fieldType)

dict_EEA_statistics = df_EEA_statistics.set_index('GEO_ID_local').to_dict()
fields_to_add.append('GEOID')
with arcpy.da.UpdateCursor(fc_block_groups, fields_to_add) as cur:
    for row in cur:
        geoid = int(row[-1])
        row[0] = dict_EEA_statistics['poverty_ratio_sum'][geoid]
        row[1] = dict_EEA_statistics['poverty_ratio_sum_share_local'][geoid]
        row[2] = dict_EEA_statistics['poverty_ratio_sum_ROC'][geoid]
        row[3] = dict_EEA_statistics['over_75_sum'][geoid]
        row[4] = dict_EEA_statistics['over_75_sum_share_local'][geoid]
        row[5] = dict_EEA_statistics['over_75_sum_ROC'][geoid]
        row[6] = dict_EEA_statistics['total_minority'][geoid]
        row[7] = dict_EEA_statistics['total_minority_share_local'][geoid]
        row[8] = dict_EEA_statistics['total_minority_ROC'][geoid]
        row[9] = dict_EEA_statistics['Total Hispanic or Latino'][geoid]
        row[10] = dict_EEA_statistics['Total Hispanic or Latino_share_local'][geoid]
        row[11] = dict_EEA_statistics['Total Hispanic or Latino_ROC'][geoid]
        row[12] = dict_EEA_statistics['do_not_speak_english_very_well'][geoid]
        row[13] = dict_EEA_statistics['do_not_speak_english_very_well_share_local'][geoid]
        row[14] = dict_EEA_statistics['do_not_speak_english_very_well_ROC'][geoid]
        row[15] = dict_EEA_statistics['Residents with a disability'][geoid]
        row[16] = dict_EEA_statistics['Residents with a disability_share_local'][geoid]
        row[17] = dict_EEA_statistics['Residents with a disability_ROC'][geoid]
        row[18] = dict_EEA_statistics['EEA_index'][geoid]
        row[19] = dict_EEA_statistics['EEA'][geoid]
        row[20] = dict_EEA_statistics['in_RN'][geoid]
        row[21] = dict_EEA_statistics['Total Population'][geoid]

        cur.updateRow(row)

Adding field poverty_ratio_pop (DOUBLE)
Adding field poverty_ratio_pop_share (DOUBLE)
Adding field poverty_ratio_ROC (DOUBLE)
Adding field over_75_pop (DOUBLE)
Adding field over_75_pop_share (DOUBLE)
Adding field over_75_ROC (DOUBLE)
Adding field total_minority_pop (DOUBLE)
Adding field total_minority_pop_share (DOUBLE)
Adding field total_minority_ROC (DOUBLE)
Adding field total_hispanic_latino_pop (DOUBLE)
Adding field total_hispanic_latino_pop_share (DOUBLE)
Adding field total_hispanic_latino_ROC (DOUBLE)
Adding field limited_english_proficiency_pop (DOUBLE)
Adding field limited_english_proficiency_pop_share (DOUBLE)
Adding field limited_english_proficiency_ROC (DOUBLE)
Adding field disability_pop (DOUBLE)
Adding field disability_pop_share (DOUBLE)
Adding field disability_ROC (DOUBLE)
Adding field eea_index (DOUBLE)
Adding field eea (TEXT)
Adding field in_RN (DOUBLE)
Adding field total_pop (DOUBLE)


## Calculate Transit Viability ##
12. Calculate Block Groups’ population density by dividing population by the Block Group area.
13. Identify each Block Group’s centroid and add a 1/4 -mile buffer to the transit stop shapefile.
14. Intersect the centroids with the transit stops buffer.
15. Intersect the result with the RN shapefile.
16. Compare each Block Group’s population density to the density of Block Groups served by transit stops. The threshold for
viability for fixed-route transit in each RN is the 10th-percentile population density of the intersected Block Groups.
17. Flag Block Groups as transit viable if population density is at least as high as the relevant RN threshold.
18. Threshold for Need for Transit Access for Equity Emphasis Areas: Transit Viable Areas are those that have a
level population density matching at least the bottom 10th percentile density of areas that are currently served by transit.

In [18]:
df_EEA_statistics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5963 entries, 0 to 5962
Data columns (total 78 columns):
 #   Column                                                                                                       Non-Null Count  Dtype  
---  ------                                                                                                       --------------  -----  
 0   NAME                                                                                                         5963 non-null   object 
 1   GEO_ID_local                                                                                                 5963 non-null   int64  
 2   Total Population                                                                                             5963 non-null   int64  
 3   Income to Poverty Level: Under 0.50                                                                          5963 non-null   int64  
 4   Income to Poverty Level: 0.50 to 0.99                       